## Training a specialised model (CNN+RNN) for Othello/Reversi

This notebook presents a new approach to estimate the next move to play in a game of Othello using Supervised Learning. The datasets come from the [Fédération Française d'Othello](https://www.ffothello.org/informatique/la-base-wthor/). The model input is on one hand the board state which will feed in a Convolutional Neural Network (CNN) and on the other hand the history of the game which will feed in a Recurrent Neural Network (RNN). The output of the model is the next move to play. We approach the task as a classification problem with a new type of kernel for the CNN : star-shaped kernel.

### Data Handling

In [110]:
import struct   # for reading the .wtb files
import os       # for file/path/directories,...  handling

import numpy as np

#### Extracting data from the WThor database

Some functions were taken or modified from the [dnnothello repo](https://github.com/wjaskowski/dnnothello/blob/master/games/othello_data.py)

The header of a .wthor file is 16 bytes long and contains the following fields:
- 1 byte: century of the file's creation
- 1 byte: year of the file's creation
- 1 byte: month of the file's creation
- 1 byte: day of the file's creation
- 4 bytes (int): number of games in the file ($\leq$ 2 147 483 648)
- 2 bytes (short): 0 here (but for other type of files : number of players, tournaments, or number of empty squares in the board ($\leq$ 65 535))
- 1 byte: year of the games
- 1 byte: size of the board {0: 8x8, 8: 8x8, 10: 10x10}
- 1 byte: 0 here the games type (1 if "solitaire", 0 otherwise)
- 1 byte: the games depth
- 1 byte: reserved

The games are stored in the file in the following format:
- 2 bytes (short): label of the tournament
- 2 bytes (short): id number of the black player
- 2 bytes (short): id number of the white player
- 1 byte: true score of the black player
- 1 byte: theoretic score of the black player

And then each move is stored as a 60 byte long record (list of moves).

In [111]:
BOARD_SIZE = 8

HEADER_LENGTH = 16
HEADER_FORMAT = "<BBBBIHHBBBB"  # Byte, Byte, Byte, Byte, Int, Short, Short, Byte, Byte, Byte, (Reserved) Byte

GAME_INFO_LENGTH = 8    
GAME_INFO_FORMAT = "<HHHBB"     # Short, Short, Short, Byte, Byte

MOVES_LENGTH = 60
MOVES_FORMAT = "<" + "B"*MOVES_LENGTH

POSSIBLE_SIZE = [0, 8]

def read_all_wtb_files(directory):
    """Generator to read all .wtb files in a directory."""
    for file_name in os.listdir(directory):
        if file_name.endswith(".wtb"):
            yield from read_wtb(os.path.join(directory, file_name))

def read_wtb(file_path):
    """Generator to read a .wtb file and yield game information and played moves."""
    with open(file_path, 'rb') as f:
        header = struct.unpack(HEADER_FORMAT, f.read(HEADER_LENGTH))
        assert header[7] in POSSIBLE_SIZE   # Check the board size
        
        for _ in range(header[4]):  # Number of games
            game_info = struct.unpack(GAME_INFO_FORMAT, f.read(GAME_INFO_LENGTH))
            played_moves = struct.unpack(MOVES_FORMAT, f.read(MOVES_LENGTH))
            yield game_info[3], played_moves    # Black player true score, moves

In [112]:
reader = read_wtb('../data/WTH_2001.wtb')
print(next(reader))

full_reader = read_all_wtb_files('../data/')
print(next(full_reader))

(11, (56, 64, 53, 46, 35, 63, 34, 66, 65, 74, 37, 43, 57, 33, 76, 24, 75, 26, 83, 36, 73, 38, 25, 16, 14, 15, 17, 47, 13, 68, 48, 58, 52, 28, 67, 23, 12, 61, 32, 42, 31, 86, 51, 41, 27, 84, 85, 82, 71, 18, 72, 11, 21, 22, 62, 81, 77, 78, 88, 87))
(34, (56, 64, 33, 36, 46, 34, 43, 67, 66, 65, 53, 63, 74, 84, 75, 57, 35, 24, 47, 38, 76, 52, 58, 37, 42, 62, 83, 82, 73, 85, 86, 87, 48, 68, 25, 14, 13, 31, 61, 51, 15, 26, 77, 23, 41, 88, 21, 72, 16, 32, 12, 22, 78, 71, 81, 11, 17, 27, 28, 18))


In [113]:
def decode_board(moves):
    """Decode a board from the 0-63 representation to the 2D representation."""
    boards = np.array([])
    player = -1  # Black starts
    for i, move in enumerate(moves):
        x, y = decode_move(move)
        # TODO Check if the move is valid. If it is not, we have to switch player.
        # TODO Add the board
        # TODO Switch player
    return boards
        
            
def decode_move(move):
    """Decode a move from the 0-63 representation to the (x, y) representation."""
    return move % 10 - 1, move // 10 - 1

def print_board(board):
    """Print a board."""
    for row in board:
        print("".join("X" if cell == 1 else "O" if cell == 2 else "." for cell in row))

In [114]:
next_moves = next(reader)[1]
np_board = decode_board(next_moves)
print_board(np_board)

0 56
1 64
2 43
3 34
4 33
5 46
6 66
7 52
8 42
9 36
10 65
11 75
12 76
13 53
14 35
15 31
16 73
17 74
18 62
19 67
20 57
21 23
22 61
23 63
24 37
25 58
26 84
27 85
28 68
29 47
30 86
31 83
32 82
33 51
34 72
35 24
36 26
37 25
38 48
39 81
40 71
41 16
42 27
43 77
44 41
45 87
46 32
47 18
48 17
49 22
50 13
51 15
52 21
53 11
54 12
55 14
56 78
57 28
58 38
59 88
OXXOOOXO
XOOOOXXO
OXXOXOXX
XXX..OOX
OOO..XXO
XXOOXXOX
XXXOOXOX
OXOXOXOO
